In [1]:
pip install selenium 4.22.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement 4.22.0 (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for 4.22.0


In [2]:
import requests
from bs4 import BeautifulSoup
import IPython
import IPython.display

import time
from selenium import webdriver
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import os.path

## load page unit test

In [3]:
query = "همراه اول"
url = f"https://www.google.com/search?q={query}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
print(url)
response = requests.get(url, headers=headers)
IPython.display.HTML(response.text)

https://www.google.com/search?q=همراه اول


In [4]:
driver = webdriver.Chrome()
#keyword = "همراه اول"
keyword = "ایرانسل"

start_date = datetime.date(2010, 1, 1)#@param {type:"string"}
end_date = datetime.date(2024, 10, 1)#@param {type:"string"}

import pandas as pd
df = pd.DataFrame()
appended_data = pd.DataFrame()

###########  while  ######################################################################################
while start_date <= end_date:
    driver.get("https://www.google.com") # ورود به صفحه گوگل
    search_box = driver.find_element("name","q")  # پیدا کردن المان مربوط به جستجو
    #search_box.send_keys("همراه اول after:2020-11-01 before:2020-12-01")  # وارد کردن عبارت مورد نظر
    search_box.send_keys('\"' + keyword + '\"' +" after:" + str(start_date) + " before:" + str(start_date + relativedelta(months=1)))  # وارد کردن عبارت مورد نظر

    time.sleep(21)#19
    search_box.send_keys(Keys.RETURN)  # ارسال جستجو با فشردن Enter
    html_content = driver.page_source # دریافت HTML صفحه جستجو شده
    time.sleep(33)#31
    button = driver.find_element("id","hdtb-tls") #tools پیدا کردن المان مربوط به 
    button.click()  #tools کلیک روی دکمه
    page_html = driver.page_source #دریافت کد اچ تی ام ال
    soup = BeautifulSoup(page_html, 'html.parser') # تحویل به پارسر

    result_stats_div = soup.find('div', {'id': 'result-stats'})
    if result_stats_div:
        result_stats_text = result_stats_div.text
        #print(result_stats_text)
        match = re.search(r'About ([\d,]+) results', result_stats_text)         # جدا کردن قسمت عددی از متن
        numeric_part = match.group(1)
        numeric_part = numeric_part.replace(',', '')     # حذف کاماها از عدد
        numeric_value = int(numeric_part)     # تبدیل رشته به عدد صحیح
        print(str(start_date) + ": " + str(numeric_value))

        df = df.assign(date=[start_date], keyword=[keyword], search_result=[numeric_value])
        appended_data = [appended_data, df]
        appended_data = pd.concat(appended_data)
        appended_data.to_csv('google-selenium.csv')
    else:
        print('تگ div با آی دی result-stats یافت نشد.')
    #print(page_html)
    start_date = start_date + relativedelta(months=+1)
appended_data = appended_data.reset_index()
print(appended_data)
appended_data.to_csv('google-selenium.csv')
driver.quit()

2010-01-01: 655
2010-02-01: 685


KeyboardInterrupt: 